# Tutorial 3.3 Thinking with Deep Learning: Week 3 Part 3
# Introduction to Deep Learning for Causal Inference on Observables

__Instructor:__ James Evans

__Notebook Author:__ Bernard Koch (https://github.com/kochbj/Deep-Learning-for-Causal-Inference)

__Notebook Editor & Teaching Assistants:__ Shiyang Lai, Avi


# Double/Debiased Machine Learning (DML)

Double/Debiased Machine Learning (DML) is a powerful method for causal inference that has gained significant attention in recent years. The method is proposed by Chernozhukov etc. in 2018 ([link](https://academic.oup.com/ectj/article/21/1/C1/5056401?login=true)). In this practical guide, we will briefly explore what DML is, how it works, and implement it using the [DoubleML](https://docs.doubleml.org/stable/index.html#doubleml-package) package.

<font color='red'><p>Please select A100 device for this colab exercise.<p></font>

In [ ]:
!pip install --upgrade scikit-learn==1.5.0 --quiet
!pip install --upgrade scikeras==0.13.0 --quiet
import scikeras
from scikeras.wrappers import KerasRegressor, KerasClassifier

# Install DoubleML package if not already installed
try:
    import doubleml
except ImportError:
    !pip install doubleml --quiet
    import doubleml

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries
import gc
import numpy as np
import pandas as pd
import sklearn
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Build LightGBM with GPU support
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import doubleml
from doubleml import DoubleMLData, DoubleMLPLR
from doubleml.datasets import make_plr_CCDDHNR2018

# Load tensorflow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set plotting styles
sns.set_theme(style="whitegrid")
face_colors = sns.color_palette('pastel')
edge_colors = sns.color_palette('dark')

In [ ]:
print("tensorflow:", tf.__version__)
print("scikit-learn:", sklearn.__version__)
print("doubleml:", doubleml.__version__)
print("scikeras:", scikeras.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## DML Basics
### Data generating process

To understand the spirit of DML, we can first think through a data-generating process (DGP) such that:

$$
y_i = \theta_0 d_i + g_0(x_i) + \zeta_i, \quad \zeta_i \sim N(0, 1),
$$

$$
d_i = m_0(x_i) + v_i, \quad v_i \sim N(0, 1),
$$

with covariates $x_i \sim N(0, \Sigma)$, where $\Sigma$ is a matrix with entries $\Sigma_{kj} = 0.7^{|j-k|}$. We are interested in performing valid inference on the causal parameter $\theta_0$. The true parameter $\theta_0$ is set to 0.5 in our simulation experiment.

Here, the nuisance functions are given by

$$
m_{0}(x_{i})=x_{i,1}+\frac{1}{4} \frac{exp\left( x_{i,3} \right)}{1+exp\left( x_{i,3} \right)},
$$

$$
g_{0}(x_{i})=\frac{exp\left( x_{i,1} \right)}{1+exp(x_{i,1})} +\frac{1}{4} x_{i,3}.
$$

We generate `n_rep` replications of the data generating process with sample size `n_obs` and compare the performance of different estimators.

In [ ]:
np.random.seed(60615)
n_rep = 1000
n_obs = 500
n_vars = 5
alpha = 0.5

data = list()

# In Python the data can be generated with doubleml.datasets.make_plr_CCDDHNR2018().
for i_rep in range(n_rep):
    (x, y, d) = make_plr_CCDDHNR2018(alpha=alpha, n_obs=n_obs, dim_x=n_vars, return_type='array')
    data.append((x, y, d))

### Regularization Bias in Simple ML-Approaches

Naive inference that is based on a direct application of machine learning methods to estimate the causal parameter $θ_0$, is generally invalid. The use of machine learning methods introduces a bias that arises due to regularization. A simple ML approach is given by randomly splitting the sample into two parts. On the auxiliary sample indexed by $i ∈ I^C$ the nuisance function $g_0(X)$ is estimated with an ML method, for example a decision tree or a MLP learner. Given the estimate $\hat{g_0}(X)$, the final estimate of $θ_0$
 is obtained as ($n=N/2$) using the other half of observations indexed with $i \in I$

 $$
 \hat{\theta_{0}} =\left( \frac{1}{n} \sum_{i\in I} D_{i}^{2} \right)^{-1} \frac{1}{n} \sum_{i\in I} D_{i}\left( Y_{i}-\hat{g_{0}} \left( X_{i} \right) \right).
 $$

As this corresponds to a “non-orthogonal” score, which is not implemented in the DoubleML package, we need to define a custom callable.

In [ ]:
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

Remark that the estimator is not able to estimate $\hat{g_0}(X)$ directly, but has to be based on a preliminary estimate of $\hat{m_0}(X)$. All following estimators with `score="IV-type"` are based on the same preliminary procedure. Furthermore, remark that we are using external predictions to avoid cross-fitting (for demonstration purposes).

In [ ]:
# Decision tree model (~10min)
np.random.seed(60615)

ml_l = LGBMRegressor(n_estimators=300, learning_rate=0.1, verbose=-1)
ml_m = LGBMRegressor(n_estimators=300, learning_rate=0.1, verbose=-1)

ml_g = clone(ml_l)

theta_nonorth_dt = np.full(n_rep, np.nan)
se_nonorth_dt = np.full(n_rep, np.nan)

for i_rep in tqdm(range(n_rep), total=n_rep):
    print(f'Replication {i_rep+1}/{n_rep}', end='\r')
    (x, y, d) = data[i_rep]

    # choose a random sample for training and estimation
    i_train, i_est = train_test_split(np.arange(n_obs), test_size=0.5, random_state=42)

    # fit the ML algorithms on the training sample
    ml_l.fit(x[i_train, :], y[i_train])
    ml_m.fit(x[i_train, :], d[i_train])

    psi_a = -np.multiply(d[i_train] - ml_m.predict(x[i_train, :]), d[i_train] - ml_m.predict(x[i_train, :]))
    psi_b = np.multiply(d[i_train] - ml_m.predict(x[i_train, :]), y[i_train] - ml_l.predict(x[i_train, :]))
    theta_initial = -np.nanmean(psi_b) / np.nanmean(psi_a)
    ml_g.fit(x[i_train, :], y[i_train] - theta_initial * d[i_train])

    # create out-of-sample predictions
    l_hat = ml_l.predict(x[i_est, :])
    m_hat = ml_m.predict(x[i_est, :])
    g_hat = ml_g.predict(x[i_est, :])

    external_predictions = {
        'd': {
            'ml_l': l_hat.reshape(-1, 1),
            'ml_m': m_hat.reshape(-1, 1),
            'ml_g': g_hat.reshape(-1, 1)
        }
    }

    obj_dml_data = DoubleMLData.from_arrays(x[i_est, :], y[i_est], d[i_est])
    obj_dml_plr_nonorth = DoubleMLPLR(obj_dml_data,
                                    ml_l, ml_m, ml_g,
                                    n_folds=2,
                                    score=non_orth_score)
    obj_dml_plr_nonorth.fit(external_predictions=external_predictions)
    theta_nonorth_dt[i_rep] = obj_dml_plr_nonorth.coef[0]
    se_nonorth_dt[i_rep] = obj_dml_plr_nonorth.se[0]

fig_non_orth, ax = plt.subplots(constrained_layout=True);
ax = sns.histplot((theta_nonorth_dt - alpha)/se_nonorth_dt,
                color=face_colors[0], edgecolor = edge_colors[0],
                stat='density', bins=30, label='Non-orthogonal DT');
ax.axvline(0., color='k');
xx = np.arange(-5, +5, 0.001)
yy = stats.norm.pdf(xx)
ax.plot(xx, yy, color='k', label='$\\mathcal{N}(0, 1)$');
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0));
ax.set_xlim([-6., 6.]);
ax.set_xlabel('$(\hat{\\theta}_0 - \\theta_0)/\hat{\sigma}$');
plt.show()

Next, we implement the same process but change the model to a one-layer multi-layer perceptron. This is intended to demonstrate that DL models also suffer from the same biases as traditional ML algorithms. You can simply re-run the two code chunks below yourself. Note that we shrink the `n_rep` significantly to 1/5, as it is extremely time-consuming.

In [ ]:
# Ensure GPU is configured
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, False)
    print("GPU is available and configured.")
else:
    print("Using CPU.")

def create_mlp_model(
    n_features,
    n_hidden=32,
    learning_rate=0.01
):
    model = Sequential()
    model.add(Dense(n_hidden, activation='relu', input_shape=(n_features,)))
    model.add(Dense(1, activation='linear'))

    model.compile(
        loss='mean_squared_error',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=[]
    )
    return model

# Early stopping to limit unnecessary epochs once validation loss plateaus
early_stop = EarlyStopping(patience=2, restore_best_weights=True)

In [ ]:
# Example: single-hidden-layer MLP (~30min)
# Build KerasRegressor pipelines using SciKeras + StandardScaler
ml_l_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", KerasRegressor(
        model=create_mlp_model,
        n_features=x.shape[1],
        n_hidden=32,            # single hidden layer of size 32
        learning_rate=0.01,
        epochs=20,
        batch_size=64,
        verbose=0,
        callbacks=[early_stop]
    ))
])

ml_m_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", KerasRegressor(
        model=create_mlp_model,
        n_features=x.shape[1],
        n_hidden=32,
        learning_rate=0.01,
        epochs=20,
        batch_size=64,
        verbose=0,
        callbacks=[early_stop]
    ))
])

ml_g_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", KerasRegressor(
        model=create_mlp_model,
        n_features=x.shape[1],
        n_hidden=32,
        learning_rate=0.01,
        epochs=20,
        batch_size=64,
        verbose=0,
        callbacks=[early_stop]
    ))
])

small_n_rep = int(n_rep / 5)
theta_nonorth_mlp = np.full(small_n_rep, np.nan)
se_nonorth_mlp    = np.full(small_n_rep, np.nan)

with tf.device('/GPU:0'):  # Ensure GPU usage in Colab
    for i_rep in tqdm(range(small_n_rep), total=small_n_rep):
        (x_i, y_i, d_i) = data[i_rep]

        i_train, i_est = train_test_split(
            np.arange(n_obs),
            test_size=0.5,
            random_state=42
        )

        # Fit ml_l and ml_m on the training sample
        ml_l_mlp.fit(x_i[i_train, :], y_i[i_train])
        ml_m_mlp.fit(x_i[i_train, :], d_i[i_train])

        # Compute initial guess for partial out
        d_hat_train = ml_m_mlp.predict(x_i[i_train, :])
        y_hat_train = ml_l_mlp.predict(x_i[i_train, :])

        psi_a = -(d_i[i_train] - d_hat_train) * (d_i[i_train] - d_hat_train)
        psi_b =  (d_i[i_train] - d_hat_train) * (y_i[i_train] - y_hat_train)
        theta_initial = -np.nanmean(psi_b) / np.nanmean(psi_a)

        # Fit ml_g on (y_i - theta_initial * d_i)
        residual_for_g = y_i[i_train] - theta_initial * d_i[i_train]
        ml_g_mlp.fit(x_i[i_train, :], residual_for_g)

        # Out-of-sample predictions
        l_hat = ml_l_mlp.predict(x_i[i_est, :])
        m_hat = ml_m_mlp.predict(x_i[i_est, :])
        g_hat = ml_g_mlp.predict(x_i[i_est, :])

        # Provide these to DoubleML
        external_predictions = {
            'd': {
                'ml_l': l_hat.reshape(-1, 1),
                'ml_m': m_hat.reshape(-1, 1),
                'ml_g': g_hat.reshape(-1, 1)
            }
        }

        # DoubleML data & fit
        obj_dml_data = DoubleMLData.from_arrays(
            x_i[i_est, :], y_i[i_est], d_i[i_est]
        )
        obj_dml_plr_nonorth = DoubleMLPLR(
            obj_dml_data,
            ml_l_mlp,
            ml_m_mlp,
            ml_g_mlp,
            n_folds=2,
            score=non_orth_score
        )
        obj_dml_plr_nonorth.fit(external_predictions=external_predictions)

        theta_nonorth_mlp[i_rep] = obj_dml_plr_nonorth.coef[0]
        se_nonorth_mlp[i_rep]    = obj_dml_plr_nonorth.se[0]

# Visualization
fig_non_orth_mlp, ax = plt.subplots(constrained_layout=True)
ax = sns.histplot(
    (theta_nonorth_mlp - alpha) / se_nonorth_mlp,
    color = face_colors[1], edgecolor = edge_colors[1],
    stat='density',
    bins=30,
    label='Non-orthogonal MLP'
)
ax.axvline(0., color='k')
xx = np.arange(-5, +5, 0.001)
yy = stats.norm.pdf(xx)
ax.plot(xx, yy, color='k', label='$\\mathcal{N}(0, 1)$')
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
ax.set_xlim([-6., 6.])
ax.set_xlabel('$(\\hat{\\theta}_0 - \\theta_0)/\\hat{\\sigma}$')
plt.show()

The regularization bias in the simple ML-approach is caused by the slow convergence of $\hat{\theta}_0$

$$
|\sqrt{n} (\hat{\theta_0} - \theta_0) | \rightarrow_{P} \infty
$$

i.e., slower than $1/\sqrt{n}$.
The driving factor is the bias that arises by learning $g$ with a random forest or any other ML technique.
A heuristic illustration is given by

$$
\sqrt{n}(\hat{\theta_0} - \theta_0) = \underbrace{\left(\frac{1}{n} \sum_{i\in I} D_i^2\right)^{-1} \frac{1}{n} \sum_{i\in I} D_i \zeta_i}_{=:a}
+  \underbrace{\left(\frac{1}{n} \sum_{i\in I} D_i^2\right)^{-1} \frac{1}{n} \sum_{i\in I} D_i (g_0(X_i) - \hat{g_0}(X_i))}_{=:b}.
$$

$a$ is approximately Gaussian under mild conditions.
However, $b$ (the regularization bias) diverges in general.

### Overcoming regularization bias by orthogonalization

To overcome the regularization bias we can partial out the effect of $X$ from $D$ to obtain the orthogonalized regressor $V = D - m(X)$. This can be achieved by setting `score="IV-type"`. We then use the final estimate

$$
\check{\theta_0} = \left(\frac{1}{n} \sum_{i\in I} \hat{V_i} D_i\right)^{-1} \frac{1}{n} \sum_{i\in I} \hat{V_i} (Y_i - \hat{g_0}(X_i)).
$$

The following figure shows the distribution of the resulting estimates $\hat{\theta}_0$ without sample-splitting. Again, we are using external predictions to avoid cross-fitting (for demonstration purposes).

In [ ]:
np.random.seed(60615)

theta_orth_nosplit_dt = np.full(n_rep, np.nan)
se_orth_nosplit_dt = np.full(n_rep, np.nan)

for i_rep in tqdm(range(n_rep), total=n_rep):
    print(f'Replication {i_rep+1}/{n_rep}', end='\r')
    (x, y, d) = data[i_rep]

    # fit the ML algorithms on the training sample
    ml_l.fit(x, y)
    ml_m.fit(x, d)

    psi_a = -np.multiply(d - ml_m.predict(x), d - ml_m.predict(x))
    psi_b = np.multiply(d - ml_m.predict(x), y - ml_l.predict(x))
    theta_initial = -np.nanmean(psi_b) / np.nanmean(psi_a)
    ml_g.fit(x, y - theta_initial * d)

    l_hat = ml_l.predict(x)
    m_hat = ml_m.predict(x)
    g_hat = ml_g.predict(x)

    external_predictions = {
        'd': {
            'ml_l': l_hat.reshape(-1, 1),
            'ml_m': m_hat.reshape(-1, 1),
            'ml_g': g_hat.reshape(-1, 1)
        }
    }

    obj_dml_data = DoubleMLData.from_arrays(x, y, d)

    obj_dml_plr_orth_nosplit = DoubleMLPLR(obj_dml_data,
                                        ml_l, ml_m, ml_g,
                                        score='IV-type')
    obj_dml_plr_orth_nosplit.fit(external_predictions=external_predictions)
    theta_orth_nosplit_dt[i_rep] = obj_dml_plr_orth_nosplit.coef[0]
    se_orth_nosplit_dt[i_rep] = obj_dml_plr_orth_nosplit.se[0]

fig_orth_nosplit, ax = plt.subplots(constrained_layout=True);
ax = sns.histplot((theta_orth_nosplit_dt - alpha)/se_orth_nosplit_dt,
                color=face_colors[2], edgecolor = edge_colors[2],
                stat='density', bins=30, label='Double ML (DT, no sample splitting)');
ax.axvline(0., color='k');
xx = np.arange(-5, +5, 0.001)
yy = stats.norm.pdf(xx)
ax.plot(xx, yy, color='k', label='$\\mathcal{N}(0, 1)$');
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0));
ax.set_xlim([-6., 6.]);
ax.set_xlabel('$(\hat{\\theta}_0 - \\theta_0)/\hat{\sigma}$');
plt.show()

We won't do deep learning model for this case because the story is the same. If the nuisance models $\hat{g_0}()$ and $\hat{m}()$ are estimated on the whole dataset, which is also used for obtaining the final estimate $\check{\theta_0}$, another bias is observed.

### Sample splitting to remove bias induced by overfitting

Using sample splitting, i.e., estimate the nuisance models $\hat{g_0}()$ and $\hat{m}()$ on one part of the data (training data) and estimate $\check{\theta}_0$ on the other part of the data (test data), overcomes the bias induced by overfitting. We can exploit the benefits of cross-fitting by switching the role of the training and test sample. Cross-fitting performs well empirically because the entire sample can be used for estimation.

The following figure shows the distribution of the resulting estimates $\hat{\theta_0}$ with orthogonal score and sample-splitting.

In [ ]:
np.random.seed(60615)

theta_dml_dt = np.full(n_rep, np.nan)
se_dml_dt = np.full(n_rep, np.nan)

for i_rep in tqdm(range(n_rep), total=n_rep):
    print(f'Replication {i_rep+1}/{n_rep}', end='\r')
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr = DoubleMLPLR(obj_dml_data,
                            ml_l, ml_m, ml_g,
                            n_folds=2,
                            score='IV-type')
    obj_dml_plr.fit()
    theta_dml_dt[i_rep] = obj_dml_plr.coef[0]
    se_dml_dt[i_rep] = obj_dml_plr.se[0]

fig_dml, ax = plt.subplots(constrained_layout=True);
ax = sns.histplot((theta_dml_dt - alpha)/se_dml_dt,
                color=face_colors[3], edgecolor = edge_colors[3],
                stat='density', bins=30, label='Double ML (DT) with cross-fitting');
ax.axvline(0., color='k');
xx = np.arange(-5, +5, 0.001)
yy = stats.norm.pdf(xx)
ax.plot(xx, yy, color='k', label='$\\mathcal{N}(0, 1)$');
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0));
ax.set_xlim([-6., 6.]);
ax.set_xlabel('$(\hat{\\theta}_0 - \\theta_0)/\hat{\sigma}$');
plt.show()

In [ ]:
# MLP (~45h; you may skip this code chunk to save your time)
ml_l_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", KerasRegressor(
        model=create_mlp_model,
        n_features=x.shape[1],
        n_hidden=32,
        learning_rate=0.01,
        epochs=20,
        batch_size=64,
        verbose=0,
        callbacks=[early_stop]
    ))
])

ml_m_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", KerasRegressor(
        model=create_mlp_model,
        n_features=x.shape[1],
        n_hidden=32,
        learning_rate=0.01,
        epochs=20,
        batch_size=64,
        verbose=0,
        callbacks=[early_stop]
    ))
])

ml_g_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", KerasRegressor(
        model=create_mlp_model,
        n_features=x.shape[1],
        n_hidden=32,
        learning_rate=0.01,
        epochs=20,
        batch_size=64,
        verbose=0,
        callbacks=[early_stop]
    ))
])

small_n_rep = int(n_rep / 5)
theta_dml_mlp = np.full(small_n_rep, np.nan)
se_dml_mlp = np.full(small_n_rep, np.nan)

for i_rep in tqdm(range(small_n_rep), total=small_n_rep):
    print(f'Replication {i_rep+1}/{small_n_rep}', end='\r')
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr = DoubleMLPLR(obj_dml_data,
                            ml_l_mlp, ml_m_mlp, ml_g_mlp,
                            n_folds=2,
                            score='IV-type')
    obj_dml_plr.fit()
    theta_dml_mlp[i_rep] = obj_dml_plr.coef[0]
    se_dml_mlp[i_rep] = obj_dml_plr.se[0]
    tf.keras.backend.clear_session()

fig_dml, ax = plt.subplots(constrained_layout=True)
ax = sns.histplot((theta_dml_mlp - alpha)/se_dml_mlp,
                color=face_colors[4], edgecolor = edge_colors[4],
                stat='density', bins=30, label='Double ML (MLP) with cross-fitting')
ax.axvline(0., color='k');
xx = np.arange(-5, +5, 0.001)
yy = stats.norm.pdf(xx)
ax.plot(xx, yy, color='k', label='$\\mathcal{N}(0, 1)$')
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
ax.set_xlim([-6., 6.])
ax.set_xlabel('$(\hat{\\theta}_0 - \\theta_0)/\hat{\sigma}$')
plt.show()

### Double/debiased machine learning

To illustrate the benefits of the auxiliary prediction step in the DML framework we write the error as

$$
\sqrt{n}(\check{\theta_0} - \theta_0) = a^* + b^* + c^*
$$

Chernozhukov et al. (2018) argues that:

The first term

$$
a^* := (EV^2)^{-1} \frac{1}{\sqrt{n}} \sum_{i\in I} V_i \zeta_i
$$

will be asymptotically normally distributed.

The second term

$$
b^* := (EV^2)^{-1} \frac{1}{\sqrt{n}} \sum_{i\in I} (\hat{m}(X_i) - m(X_i)) (\hat{g_0}(X_i) - g_0(X_i))
$$

vanishes asymptotically for many data generating processes.

The third term $c^*$ vanishes in probability if sample splitting is applied.

## DML Example: Impact of 401(k) on Financial Wealth

In this real-data example, we illustrate how the [DoubleML](https://docs.doubleml.org/stable/index.html) package can be used to estimate the effect of 401(k) eligibility and participation on accumulated assets. The 401(k) data set has been analyzed in several studies, among others [Chernozhukov et al. (2018)](https://arxiv.org/abs/1608.00060).

401(k) plans are pension accounts sponsored by employers. The key problem in determining the effect of participation in 401(k) plans on accumulated assets is saver heterogeneity coupled with the fact that the decision to enroll in a 401(k) is non-random. It is generally recognized that some people have a higher preference for saving than others. It also seems likely that those individuals with high unobserved preference for saving would be most likely to choose to participate in tax-advantaged retirement savings plans and would tend to have otherwise high amounts of accumulated assets. The presence of unobserved savings preferences with these properties then implies that conventional estimates that do not account for saver heterogeneity and endogeneity of participation will be biased upward, tending to overstate the savings effects of 401(k) participation.

One can argue that eligibility for enrolling in a 401(k) plan in this data can be taken as exogenous after conditioning on a few observables of which the most important for their argument is income. The basic idea is that, at least around the time 401(k)’s initially became available, people were unlikely to be basing their employment decisions on whether an employer offered a 401(k) but would instead focus on income and other aspects of the job.

### Data

The preprocessed data can be fetched by calling [fetch_401K()](https://docs.doubleml.org/stable/api/generated/doubleml.datasets.fetch_401K.html#doubleml.datasets.fetch_401K). Note that an internet connection is required for loading the data.

In [ ]:
from doubleml.datasets import fetch_401K

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from xgboost import XGBClassifier, XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = 10., 7.5
sns.set(font_scale=1.5)
sns.set_style('whitegrid', {'axes.spines.top': False,
                            'axes.spines.bottom': False,
                            'axes.spines.left': False,
                            'axes.spines.right': False})

# load the 401k dataset
data = fetch_401K(return_type='DataFrame')

In [ ]:
# Temporary fix for https://github.com/DoubleML/doubleml-docs/issues/45 / https://github.com/scikit-learn/scikit-learn/issues/21997
# Can be removed when scikit-learn version 1.2.0 is released
dtypes = data.dtypes
dtypes['nifa'] = 'float64'
dtypes['net_tfa'] = 'float64'
dtypes['tw'] = 'float64'
dtypes['inc'] = 'float64'
data = data.astype(dtypes)

data.head()

In [ ]:
data.describe()

The data consist of 9,915 observations at the household level drawn from the 1991 Survey of Income and Program Participation (SIPP).  All the variables are referred to 1990. We use net financial assets (*net\_tfa*) as the outcome variable, $Y$,  in our analysis. The net financial assets are computed as the sum of IRA balances, 401(k) balances, checking accounts, saving bonds, other interest-earning accounts, other interest-earning assets, stocks, and mutual funds less non mortgage debts.

Among the $9915$ individuals, $3682$ are eligible to participate in the program. The variable *e401* indicates eligibility and *p401* indicates participation, respectively.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)  # 1 row, 2 columns

# Plot the first bar plot for e401
data['e401'].value_counts().plot(kind='bar', color=face_colors, ax=axes[0])
axes[0].set_title('Eligibility, 401(k)')
axes[0].set_xlabel('e401')
axes[0].set_ylabel('Count')

# Plot the second bar plot for p401
data['p401'].value_counts().plot(kind='bar', color=face_colors, ax=axes[1])
axes[1].set_title('Participation, 401(k)')
axes[1].set_xlabel('p401')
axes[1].set_ylabel('')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

# Plot for e401 = 0
sns.kdeplot(data=data[data["e401"] == 0], x="net_tfa", fill=True,
            color=face_colors[0], ax=axes[0])
axes[0].set_title("e401 = 0", fontsize=16)
axes[0].set_xlabel("Net Total Financial Assets (net_tfa)", fontsize=14)
axes[0].set_ylabel("Density", fontsize=14)
axes[0].tick_params(labelsize=12)

# Plot for e401 = 1
sns.kdeplot(data=data[data["e401"] == 1], x="net_tfa", fill=True,
            color=face_colors[1], ax=axes[1])
axes[1].set_title("e401 = 1", fontsize=16)
axes[1].set_xlabel("Net Total Financial Assets (net_tfa)", fontsize=14)
axes[1].tick_params(labelsize=10)

plt.tight_layout()
plt.show()

As a first estimate, we calculate the unconditional average predictive effect (APE) of 401(k) eligibility on accumulated assets. This effect corresponds to the average treatment effect if 401(k) eligibility would be assigned to individuals in an entirely randomized way. The unconditional APE of e401 is about $19559$:

In [ ]:
data[['e401', 'net_tfa']].groupby('e401').mean().diff()

Among the $3682$ individuals that  are eligible, $2594$ decided to participate in the program. The unconditional APE of p401 is about $27372$:

In [ ]:
data[['p401', 'net_tfa']].groupby('p401').mean().diff()

As discussed, these estimates are biased since they do not account for saver heterogeneity and endogeneity of participation.

Let's use the package [DoubleML](https://docs.doubleml.org/stable/index.html) to estimate the average treatment effect of 401(k) eligibility, i.e. `e401`, and participation, i.e. `p401`, on net financial assets `net_tfa`.

### Estimating the Average Treatment Effect of 401(k) Eligibility on Net Financial Assets

We first look at the treatment effect of `e401` on net total financial assets. We give estimates of the ATE in the linear model

$$
\begin{equation*}
Y = D \alpha + f(X)'\beta+ \epsilon,
\end{equation*}
$$

where $f(X)$ is a dictonary applied to the raw regressors. $X$ contains variables on marital status, two-earner status, defined benefit pension status, IRA participation, home ownership, family size, education, age, and income.

In the following, we will consider two different models,

* a basic model specification that includes the raw regressors, i.e., $f(X) = X$, and

* a flexible model specification, where $f(X)$ includes the raw regressors $X$ and the orthogonal polynomials of degree 2 for the variables family size education, age, and income.

We will use the basic model specification whenever we use nonlinear methods, for example regression trees or random forests, and use the flexible model for linear methods such as the lasso. There are, of course, multiple ways how the model can be specified even more flexibly, for example including interactions of variable and higher order interaction. However, for the sake of simplicity we stick to the specification above. Users who are interested in varying the model can adapt the code below accordingly, for example to implement the orignal specification in [Chernozhukov et al. (2018)](https://arxiv.org/abs/1608.00060).

In the first step, we report estimates of the average treatment effect (ATE) of 401(k) eligibility on net financial assets both in the partially linear regression (PLR) model and in the interactive regression model (IRM) allowing for heterogeneous treatment effects.

#### The Data Backend: `DoubleMLData`

To start our analysis, we initialize the data backend, i.e., a new instance of a [DoubleMLData](https://docs.doubleml.org/dev/api/generated/doubleml.DoubleMLData.html#doubleml.DoubleMLData) object. We implement the regression model by using scikit-learn's `PolynomialFeatures` class.

To implement both models (basic and flexible), we generate two data backends: `data_dml_base` and `data_dml_flex`.

In [ ]:
# Set up basic model: Specify variables for data-backend
features_base = ['age', 'inc', 'educ', 'fsize', 'marr',
                 'twoearn', 'db', 'pira', 'hown']

# Initialize DoubleMLData (data-backend of DoubleML)
data_dml_base = doubleml.DoubleMLData(data,
                                 y_col='net_tfa',
                                 d_cols='e401',
                                 x_cols=features_base)

print(data_dml_base)

In [ ]:
# Set up a model according to regression formula with polynomials
features = data.copy()[['marr', 'twoearn', 'db', 'pira', 'hown']]

poly_dict = {'age': 2,
             'inc': 2,
             'educ': 2,
             'fsize': 2}
for key, degree in poly_dict.items():
    poly = PolynomialFeatures(degree, include_bias=False)
    data_transf = poly.fit_transform(data[[key]])
    x_cols = poly.get_feature_names_out([key])
    data_transf = pd.DataFrame(data_transf, columns=x_cols)

    features = pd.concat((features, data_transf),
                          axis=1, sort=False)
model_data = pd.concat((data.copy()[['net_tfa', 'e401']], features.copy()),
                        axis=1, sort=False)

# Initialize DoubleMLData (data-backend of DoubleML)
data_dml_flex = doubleml.DoubleMLData(model_data, y_col='net_tfa', d_cols='e401')

print(data_dml_flex)

#### Partially Linear Regression Model (PLR)

We start using lasso to estimate the function $g_0$ and $m_0$ in the following PLR model:

$$
\begin{aligned}
& Y = D\theta_0 + g_0(X) + \zeta, &\quad E[\zeta \mid D,X]= 0,\\
& D = m_0(X) +  V, &\quad E[V \mid X] = 0.
\end{aligned}
$$

To estimate the causal parameter $\theta_0$ here, we use double machine learning with 3-fold cross-fitting.

Estimation of the nuisance components $g_0$ and $m_0$, is based on the lasso with cross-validated choice of the penalty term , $\lambda$, as provided by [scikit-learn](https://scikit-learn.org). We load the learner by initializing instances from the classes [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and [LogisticRegressionCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html). Hyperparameters and options can be set during instantiation of the learner. Here we specify that the lasso should use that value of $\lambda$ that minimizes the cross-validated mean squared error which is based on 5-fold cross validation.

We start by estimation the ATE in the basic model and then repeat the estimation in the flexible model.

In [ ]:
# Initialize learners
Cs = 0.0001*np.logspace(0, 4, 10)
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5, max_iter=10000))
lasso_class = make_pipeline(StandardScaler(),
                            LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear',
                                                 Cs = Cs, max_iter=1000))

np.random.seed(60615)
# Initialize DoubleMLPLR model
dml_plr_lasso = doubleml.DoubleMLPLR(data_dml_base,
                                ml_l = lasso,
                                ml_m = lasso_class,
                                n_folds = 3)

dml_plr_lasso.fit(store_predictions=True)
print(dml_plr_lasso.summary)

In [ ]:
# Estimate the ATE in the flexible model with lasso
np.random.seed(60615)
dml_plr_lasso = doubleml.DoubleMLPLR(data_dml_flex,
                                ml_l = lasso,
                                ml_m = lasso_class,
                                n_folds = 3)

dml_plr_lasso.fit(store_predictions=True)
lasso_summary = dml_plr_lasso.summary

print(lasso_summary)

Alternatively, we can repeat this procedure with other machine learning methods, for example a random forest learner as provided by the [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) class in [scikit-learn](https://scikit-learn.org).

In [ ]:
# Random Forest
randomForest = RandomForestRegressor(
    n_estimators=500, max_depth=7, max_features=3, min_samples_leaf=3)
randomForest_class = RandomForestClassifier(
    n_estimators=500, max_depth=5, max_features=4, min_samples_leaf=7)

np.random.seed(60615)
dml_plr_forest = doubleml.DoubleMLPLR(data_dml_base,
                                 ml_l = randomForest,
                                 ml_m = randomForest_class,
                                 n_folds = 3)
dml_plr_forest.fit(store_predictions=True)
forest_summary = dml_plr_forest.summary

print(forest_summary)

In [ ]:
# Trees
trees = DecisionTreeRegressor(
    max_depth=30, ccp_alpha=0.0047, min_samples_split=203, min_samples_leaf=67)
trees_class = DecisionTreeClassifier(
    max_depth=30, ccp_alpha=0.0042, min_samples_split=104, min_samples_leaf=34)

np.random.seed(60615)
dml_plr_tree = doubleml.DoubleMLPLR(data_dml_base,
                               ml_l = trees,
                               ml_m = trees_class,
                               n_folds = 3)
dml_plr_tree.fit(store_predictions=True)
tree_summary = dml_plr_tree.summary

print(tree_summary)

In [ ]:
# Boosted Trees
boost = XGBRegressor(n_jobs=1, objective = "reg:squarederror",
                     eta=0.1, n_estimators=35)
boost_class = XGBClassifier(use_label_encoder=False, n_jobs=1,
                            objective = "binary:logistic", eval_metric = "logloss",
                            eta=0.1, n_estimators=34)

np.random.seed(60615)
dml_plr_boost = doubleml.DoubleMLPLR(data_dml_base,
                                ml_l = boost,
                                ml_m = boost_class,
                                n_folds = 3)
dml_plr_boost.fit(store_predictions=True)
boost_summary = dml_plr_boost.summary

print(boost_summary)

In [ ]:
# MLP
n_features = data_dml_base.x.shape[1]

# 2. Define MLP for regression (outcome)
#    Omit explicit input_shape and let SciKeras handle shape inference
def create_mlp_model1_regression(n_features=10, n_hidden=32, learning_rate=0.01, **kwargs):
    model = Sequential()
    model.add(Dense(n_hidden, activation='relu', input_shape=(n_features,)))
    model.add(Dense(1, activation='linear'))
    model.compile(
        loss='mean_squared_error',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=[]
    )
    return model

# 3. Define MLP for classification (treatment)
#    If your treatment is truly binary (0/1).
def create_mlp_model1_classification(n_features=10, n_hidden=32, learning_rate=0.01, **kwargs):
    model = Sequential()
    model.add(Dense(n_hidden, activation='relu', input_shape=(n_features,)))
    # Binary classification => 1 output node, sigmoid activation
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )
    return model

# 4. Wrap each with SciKeras
ml_l_mlp = KerasRegressor(
    model=create_mlp_model1_regression,
    model__n_features=n_features,     # match data_dml_base.x.shape[1]
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

ml_m_mlp = KerasClassifier(
    model=create_mlp_model1_classification,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

# 5. Create the DoubleML PLR object
dml_plr_mlp = DoubleMLPLR(
    data_dml_base,
    ml_l=ml_l_mlp,  # MLP regressor for the outcome
    ml_m=ml_m_mlp,  # MLP classifier for the binary treatment
    n_folds=3
    # Optionally: score='IV-type' or 'partialling out', etc.
)

# 6. Fit
dml_plr_mlp.fit(store_predictions=True)

# 7. Show summary
mlp_summary = dml_plr_mlp.summary
print(mlp_summary)

In [ ]:
def create_mlp_model2_regression(
    n_features=10,
    n_hidden1=64,
    n_hidden2=32,
    learning_rate=0.01,
    dropout_rate=0.2,
    **kwargs
):
    model = Sequential()
    # First hidden layer
    model.add(Dense(n_hidden1, activation='relu', input_shape=(n_features,)))
    model.add(Dropout(dropout_rate))
    # Second hidden layer
    model.add(Dense(n_hidden2, activation='relu'))
    # Output layer for regression
    model.add(Dense(1, activation='linear'))
    model.compile(
        loss='mean_squared_error',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=[]
    )
    return model

def create_mlp_model2_classification(
    n_features=10,
    n_hidden1=64,
    n_hidden2=32,
    learning_rate=0.01,
    dropout_rate=0.2,
    **kwargs
):
    model = Sequential()
    # First hidden layer
    model.add(Dense(n_hidden1, activation='relu', input_shape=(n_features,)))
    model.add(Dropout(dropout_rate))
    # Second hidden layer
    model.add(Dense(n_hidden2, activation='relu'))
    # Output layer (sigmoid for binary classification)
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )
    return model

ml_l_mlp = KerasRegressor(
    model=create_mlp_model2_regression,
    model__n_features=n_features,
    model__n_hidden1=64,
    model__n_hidden2=32,
    model__dropout_rate=0.2,     # Increase or decrease as needed
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

ml_m_mlp = KerasClassifier(
    model=create_mlp_model2_classification,
    model__n_features=n_features,
    model__n_hidden1=64,
    model__n_hidden2=32,
    model__dropout_rate=0.2,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

dml_plr_mlp2 = DoubleMLPLR(
    data_dml_base,
    ml_l=ml_l_mlp,   # Deeper MLP regressor for outcome
    ml_m=ml_m_mlp,   # Deeper MLP classifier for binary treatment
    n_folds=3        # Cross-fitting folds
    # score=...       # e.g., 'partialling out', 'IV-type', etc.
)

dml_plr_mlp2.fit(store_predictions=True)

mlp2_summary = dml_plr_mlp2.summary
print(mlp2_summary)

Let's sum up the results:

In [ ]:
plr_summary = pd.concat((lasso_summary, forest_summary, tree_summary, boost_summary, mlp_summary, mlp2_summary))
plr_summary.index = ['lasso', 'forest', 'tree', 'xgboost', '1-layer mlp', '2-layer mlp']
print(plr_summary[['coef', '2.5 %', '97.5 %']])

In [ ]:
errors = np.full((2, plr_summary.shape[0]), np.nan)
errors[0, :] = plr_summary['coef'] - plr_summary['2.5 %']
errors[1, :] = plr_summary['97.5 %'] - plr_summary['coef']
plt.errorbar(plr_summary.index, plr_summary.coef, fmt='o', yerr=errors)
plt.ylim([0, 15000])

plt.title('Partially Linear Regression Model (PLR)')
plt.xlabel('ML method')
_ =  plt.ylabel('Coefficients and 95%-CI')

These estimates that flexibly account for confounding are
substantially attenuated relative to the baseline estimate (*19559*) that does not account for confounding. They suggest much smaller causal effects of 401(k) eligiblity on financial asset holdings.

#### Interactive Regression Model (IRM)

Next, we consider estimation of average treatment effects when treatment effects are fully heterogeneous:

$$
\begin{aligned}
& Y = g_0(D,X) + U, &\quad E[U\mid X,D] = 0,\\
& D = m_0(X) + V, &\quad E[V\mid X] = 0.
\end{aligned}
$$

To reduce the disproportionate impact of extreme propensity score weights in the interactive model
we trim the propensity scores which are close to the bounds.

In [ ]:
# Lasso
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5, max_iter=20000))

# Initialize DoubleMLIRM model
np.random.seed(60615)
dml_irm_lasso = doubleml.DoubleMLIRM(data_dml_flex,
                          ml_g = lasso,
                          ml_m = lasso_class,
                          trimming_threshold = 0.01,
                          n_folds = 3)
dml_irm_lasso.fit(store_predictions=True)
lasso_summary = dml_irm_lasso.summary

# Random Forest
randomForest = RandomForestRegressor(n_estimators=500)
randomForest_class = RandomForestClassifier(n_estimators=500)

np.random.seed(60615)
dml_irm_forest = doubleml.DoubleMLIRM(data_dml_base,
                                 ml_g = randomForest,
                                 ml_m = randomForest_class,
                                 trimming_threshold = 0.01,
                                 n_folds = 3)

# Set nuisance-part specific parameters
dml_irm_forest.set_ml_nuisance_params('ml_g0', 'e401', {
    'max_depth': 6, 'max_features': 4, 'min_samples_leaf': 7})
dml_irm_forest.set_ml_nuisance_params('ml_g1', 'e401', {
    'max_depth': 6, 'max_features': 3, 'min_samples_leaf': 5})
dml_irm_forest.set_ml_nuisance_params('ml_m', 'e401', {
    'max_depth': 6, 'max_features': 3, 'min_samples_leaf': 6})

dml_irm_forest.fit(store_predictions=True)
forest_summary = dml_irm_forest.summary

# Trees
trees = DecisionTreeRegressor(max_depth=30)
trees_class = DecisionTreeClassifier(max_depth=30)

np.random.seed(60615)
dml_irm_tree = doubleml.DoubleMLIRM(data_dml_base,
                               ml_g = trees,
                               ml_m = trees_class,
                               trimming_threshold = 0.01,
                               n_folds = 3)

# Set nuisance-part specific parameters
dml_irm_tree.set_ml_nuisance_params('ml_g0', 'e401', {
    'ccp_alpha': 0.0016, 'min_samples_split': 74, 'min_samples_leaf': 24})
dml_irm_tree.set_ml_nuisance_params('ml_g1', 'e401', {
    'ccp_alpha': 0.0018, 'min_samples_split': 70, 'min_samples_leaf': 23})
dml_irm_tree.set_ml_nuisance_params('ml_m', 'e401', {
    'ccp_alpha': 0.0028, 'min_samples_split': 167, 'min_samples_leaf': 55})

dml_irm_tree.fit(store_predictions=True)
tree_summary = dml_irm_tree.summary

# Boosted Trees
boost = XGBRegressor(n_jobs=1, objective = "reg:squarederror")
boost_class = XGBClassifier(use_label_encoder=False, n_jobs=1,
                            objective = "binary:logistic", eval_metric = "logloss")

np.random.seed(60615)
dml_irm_boost = doubleml.DoubleMLIRM(data_dml_base,
                                ml_g = boost,
                                ml_m = boost_class,
                                trimming_threshold = 0.01,
                                n_folds = 3)

# Set nuisance-part specific parameters
dml_irm_boost.set_ml_nuisance_params('ml_g0', 'e401', {
    'eta': 0.1, 'n_estimators': 8})
dml_irm_boost.set_ml_nuisance_params('ml_g1', 'e401', {
    'eta': 0.1, 'n_estimators': 29})
dml_irm_boost.set_ml_nuisance_params('ml_m', 'e401', {
    'eta': 0.1, 'n_estimators': 23})

dml_irm_boost.fit(store_predictions=True)
boost_summary = dml_irm_boost.summary

# 1 layer mlp
ml_g_mlp = KerasRegressor(
    model=create_mlp_model1_regression,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

ml_m_mlp = KerasClassifier(
    model=create_mlp_model1_classification,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

ml_r_mlp = KerasClassifier(
    model=create_mlp_model1_classification,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

np.random.seed(60615)
dml_irm_mlp1 = doubleml.DoubleMLIRM(
    data_dml_base,
    ml_g=ml_g_mlp,  # MLP regressor for outcome
    ml_m=ml_m_mlp,  # MLP classifier for binary treatment
    trimming_threshold=0.01,
    n_folds=3
)

# For example:
# dml_irm_mlp1.set_ml_nuisance_params('ml_g0', 'p401', {
#     'epochs': 15,
#     'model__learning_rate': 0.005
# })
# dml_irm_mlp1.set_ml_nuisance_params('ml_m', 'p401', {
#     'epochs': 25,
#     'batch_size': 64,
#     'model__n_hidden': 64
# })
# dml_irm_mlp1.set_ml_nuisance_params('ml_r1', 'p401', {
#     'epochs': 30,
#     'model__n_hidden': 16
# })

dml_irm_mlp1.fit(store_predictions=True)
mlp1_summary = dml_irm_mlp1.summary

# 2-layer MLP
ml_g_mlp = KerasRegressor(
    model=create_mlp_model2_regression,
    n_features=n_features,
    n_hidden1=64,      # or tune
    n_hidden2=32,
    dropout_rate=0.2,
    learning_rate=0.01,
    epochs=50,
    batch_size=64,
    verbose=0
)

ml_m_mlp = KerasClassifier(
    model=create_mlp_model2_classification,
    n_features=n_features,
    n_hidden1=64,
    n_hidden2=32,
    dropout_rate=0.2,
    learning_rate=0.01,
    epochs=50,
    batch_size=64,
    verbose=0
)

ml_r_mlp = KerasClassifier(
    model=create_mlp_model2_classification,
    n_features=n_features,
    n_hidden1=64,
    n_hidden2=32,
    dropout_rate=0.2,
    learning_rate=0.01,
    epochs=50,
    batch_size=64,
    verbose=0
)

np.random.seed(60615)
dml_irm_mlp2 = doubleml.DoubleMLIRM(
    data_dml_base,
    ml_g=ml_g_mlp,
    ml_m=ml_m_mlp,
    trimming_threshold=0.01,
    n_folds=3
)


# (Optional) set nuisance-part-specific parameters:
# dml_iivm_mlp.set_ml_nuisance_params('ml_g0', 'p401', {
#     'epochs': 15,
#     'model__learning_rate': 0.005
# })
# dml_iivm_mlp.set_ml_nuisance_params('ml_g1', 'p401', {
#     'epochs': 25,
#     'batch_size': 32
# })
# dml_iivm_mlp.set_ml_nuisance_params('ml_m', 'p401', {
#     'model__dropout_rate': 0.3
# })
# dml_iivm_mlp.set_ml_nuisance_params('ml_r1', 'p401', {
#     'epochs': 30,
#     'model__n_hidden1': 128
# })

dml_irm_mlp2.fit(store_predictions=True)
mlp2_summary = dml_irm_mlp2.summary

In [ ]:
irm_summary = pd.concat((lasso_summary, forest_summary, tree_summary, boost_summary, mlp1_summary, mlp2_summary))
irm_summary.index = ['lasso', 'forest', 'tree', 'xgboost', '1-layer mlp', '2-layer mlp']
print(irm_summary[['coef', '2.5 %', '97.5 %']])

In [ ]:
errors = np.full((2, irm_summary.shape[0]), np.nan)
errors[0, :] = irm_summary['coef'] - irm_summary['2.5 %']
errors[1, :] = irm_summary['97.5 %'] - irm_summary['coef']
plt.errorbar(irm_summary.index, irm_summary.coef, fmt='o', yerr=errors)
plt.ylim([0, 12500])

plt.title('Interactive Regression Model (IRM)')
plt.xlabel('ML method')
_ = plt.ylabel('Coefficients and 95%-CI')

### Local Average Treatment Effects of 401(k) Participation on Net Financial Assets

#### Interactive IV Model (IIVM)

In the examples above, we estimated the average treatment effect of *eligibility* on financial asset holdings. Now, we consider estimation of local average treatment effects (LATE) of *participation* using eligibility as an instrument for the participation decision. Under appropriate assumptions, the LATE identifies the treatment effect for so-called compliers, i.e., individuals who would only participate if eligible and otherwise not participate in the program.

As before, $Y$ denotes the outcome `net_tfa`, and $X$ is the vector of covariates. We use `e401` as a binary instrument for the treatment variable `p401`. Here the structural equation model is:

$$
\begin{aligned}
& Y = g_0(Z,X) + U, &\quad E[U\mid Z,X] = 0,\\
& D = r_0(Z,X) + V, &\quad E[V\mid Z, X] = 0,\\
& Z = m_0(X) + \zeta, &\quad E[\zeta \mid X] = 0.
\end{aligned}
$$

In [ ]:
# Initialize DoubleMLData with an instrument

# Basic model
data_dml_base_iv = doubleml.DoubleMLData(data,
                                    y_col='net_tfa',
                                    d_cols='p401',
                                    z_cols='e401',
                                    x_cols=features_base)

In [ ]:
# Flexible model
model_data = pd.concat((data.copy()[['net_tfa', 'e401', 'p401']], features.copy()),
                        axis=1, sort=False)

data_dml_iv_flex = doubleml.DoubleMLData(model_data,
                                    y_col='net_tfa',
                                    d_cols='p401',
                                    z_cols='e401')

In [ ]:
# Lasso
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5, max_iter=20000))

# Initialize DoubleMLIRM model
np.random.seed(60615)
dml_iivm_lasso = doubleml.DoubleMLIIVM(data_dml_iv_flex,
                                  ml_g = lasso,
                                  ml_m = lasso_class,
                                  ml_r = lasso_class,
                                  subgroups = {'always_takers': False,
                                             'never_takers': True},
                                  trimming_threshold = 0.01,
                                  n_folds = 3)
dml_iivm_lasso.fit(store_predictions=True)
lasso_summary = dml_iivm_lasso.summary

# Random Forest
randomForest = RandomForestRegressor(n_estimators=500)
randomForest_class = RandomForestClassifier(n_estimators=500)

np.random.seed(60615)
dml_iivm_forest = doubleml.DoubleMLIIVM(data_dml_base_iv,
                                   ml_g = randomForest,
                                   ml_m = randomForest_class,
                                   ml_r = randomForest_class,
                                   subgroups = {'always_takers': False,
                                                'never_takers': True},
                                   trimming_threshold = 0.01,
                                   n_folds = 3)

# Set nuisance-part specific parameters
dml_iivm_forest.set_ml_nuisance_params('ml_g0', 'p401', {
    'max_depth': 6, 'max_features': 4, 'min_samples_leaf': 7})
dml_iivm_forest.set_ml_nuisance_params('ml_g1', 'p401', {
    'max_depth': 6, 'max_features': 3, 'min_samples_leaf': 5})
dml_iivm_forest.set_ml_nuisance_params('ml_m', 'p401', {
    'max_depth': 6, 'max_features': 3, 'min_samples_leaf': 6})
dml_iivm_forest.set_ml_nuisance_params('ml_r1', 'p401', {
    'max_depth': 4, 'max_features': 7, 'min_samples_leaf': 6})

dml_iivm_forest.fit(store_predictions=True)
forest_summary = dml_iivm_forest.summary

# Trees
trees = DecisionTreeRegressor(max_depth=30)
trees_class = DecisionTreeClassifier(max_depth=30)

np.random.seed(60615)
dml_iivm_tree = doubleml.DoubleMLIIVM(data_dml_base_iv,
                                 ml_g = trees,
                                 ml_m = trees_class,
                                 ml_r = trees_class,
                                 subgroups = {'always_takers': False,
                                              'never_takers': True},
                                 trimming_threshold = 0.01,
                                 n_folds = 3)

# Set nuisance-part specific parameters
dml_iivm_tree.set_ml_nuisance_params('ml_g0', 'p401', {
    'ccp_alpha': 0.0016, 'min_samples_split': 74, 'min_samples_leaf': 24})
dml_iivm_tree.set_ml_nuisance_params('ml_g1', 'p401', {
    'ccp_alpha': 0.0018, 'min_samples_split': 70, 'min_samples_leaf': 23})
dml_iivm_tree.set_ml_nuisance_params('ml_m', 'p401', {
    'ccp_alpha': 0.0028, 'min_samples_split': 167, 'min_samples_leaf': 55})
dml_iivm_tree.set_ml_nuisance_params('ml_r1', 'p401', {
    'ccp_alpha': 0.0576, 'min_samples_split': 55, 'min_samples_leaf': 18})

dml_iivm_tree.fit(store_predictions=True)
tree_summary = dml_iivm_tree.summary

# Boosted Trees
boost = XGBRegressor(n_jobs=1, objective = "reg:squarederror")
boost_class = XGBClassifier(use_label_encoder=False, n_jobs=1,
                            objective = "binary:logistic", eval_metric = "logloss")

np.random.seed(60615)
dml_iivm_boost = doubleml.DoubleMLIIVM(data_dml_base_iv,
                                  ml_g = boost,
                                  ml_m = boost_class,
                                  ml_r = boost_class,
                                  subgroups = {'always_takers': False,
                                               'never_takers': True},
                                  trimming_threshold = 0.01,
                                  n_folds = 3)

# Set nuisance-part specific parameters
dml_iivm_boost.set_ml_nuisance_params('ml_g0', 'p401', {
    'eta': 0.1, 'n_estimators': 9})
dml_iivm_boost.set_ml_nuisance_params('ml_g1', 'p401', {
    'eta': 0.1, 'n_estimators': 33})
dml_iivm_boost.set_ml_nuisance_params('ml_m', 'p401', {
    'eta': 0.1, 'n_estimators': 12})
dml_iivm_boost.set_ml_nuisance_params('ml_r1', 'p401', {
    'eta': 0.1, 'n_estimators': 25})

dml_iivm_boost.fit(store_predictions=True)
boost_summary = dml_iivm_boost.summary

# 1-layer MLP
ml_g_mlp = KerasRegressor(
    model=create_mlp_model1_regression,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

ml_m_mlp = KerasClassifier(
    model=create_mlp_model1_classification,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

ml_r_mlp = KerasClassifier(
    model=create_mlp_model1_classification,
    model__n_features=n_features,
    model__n_hidden=32,
    model__learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)

np.random.seed(60615)
dml_iivm_mlp1 = doubleml.DoubleMLIIVM(
    data_dml_base_iv,
    ml_g=ml_g_mlp,  # MLP regressor for outcome
    ml_m=ml_m_mlp,  # MLP classifier for binary treatment
    ml_r=ml_r_mlp,  # MLP classifier for binary instrument propensity
    subgroups={'always_takers': False,
               'never_takers': True},
    trimming_threshold=0.01,
    n_folds=3
)

# For example:
# dml_iivm_mlp1.set_ml_nuisance_params('ml_g0', 'p401', {
#     'epochs': 15,
#     'model__learning_rate': 0.005
# })
# dml_iivm_mlp1.set_ml_nuisance_params('ml_m', 'p401', {
#     'epochs': 25,
#     'batch_size': 64,
#     'model__n_hidden': 64
# })
# dml_iivm_mlp1.set_ml_nuisance_params('ml_r1', 'p401', {
#     'epochs': 30,
#     'model__n_hidden': 16
# })

dml_iivm_mlp1.fit(store_predictions=True)
mlp1_summary = dml_iivm_mlp1.summary

# 2-layer MLP
ml_g_mlp = KerasRegressor(
    model=create_mlp_model2_regression,
    n_features=n_features,
    n_hidden1=64,      # or tune
    n_hidden2=32,
    dropout_rate=0.2,
    learning_rate=0.01,
    epochs=50,
    batch_size=64,
    verbose=0
)

ml_m_mlp = KerasClassifier(
    model=create_mlp_model2_classification,
    n_features=n_features,
    n_hidden1=64,
    n_hidden2=32,
    dropout_rate=0.2,
    learning_rate=0.01,
    epochs=50,
    batch_size=64,
    verbose=0
)

ml_r_mlp = KerasClassifier(
    model=create_mlp_model2_classification,
    n_features=n_features,
    n_hidden1=64,
    n_hidden2=32,
    dropout_rate=0.2,
    learning_rate=0.01,
    epochs=50,
    batch_size=64,
    verbose=0
)

np.random.seed(60615)
dml_iivm_mlp2 = doubleml.DoubleMLIIVM(
    data_dml_base_iv,
    ml_g=ml_g_mlp,
    ml_m=ml_m_mlp,
    ml_r=ml_r_mlp,
    subgroups={'always_takers': False,
               'never_takers': True},
    trimming_threshold=0.01,
    n_folds=3
)


# (Optional) set nuisance-part-specific parameters:
# dml_iivm_mlp.set_ml_nuisance_params('ml_g0', 'p401', {
#     'epochs': 15,
#     'model__learning_rate': 0.005
# })
# dml_iivm_mlp.set_ml_nuisance_params('ml_g1', 'p401', {
#     'epochs': 25,
#     'batch_size': 32
# })
# dml_iivm_mlp.set_ml_nuisance_params('ml_m', 'p401', {
#     'model__dropout_rate': 0.3
# })
# dml_iivm_mlp.set_ml_nuisance_params('ml_r1', 'p401', {
#     'epochs': 30,
#     'model__n_hidden1': 128
# })

dml_iivm_mlp2.fit(store_predictions=True)
mlp2_summary = dml_iivm_mlp2.summary

In [ ]:
iivm_summary = pd.concat((lasso_summary, forest_summary, tree_summary, boost_summary, mlp1_summary, mlp2_summary))
iivm_summary.index = ['lasso', 'forest', 'tree', 'xgboost', '1-layer mlp', '2-layer mlp']
print(iivm_summary[['coef', '2.5 %', '97.5 %']])

In [ ]:
errors = np.full((2, iivm_summary.shape[0]), np.nan)
errors[0, :] = iivm_summary['coef'] - iivm_summary['2.5 %']
errors[1, :] = iivm_summary['97.5 %'] - iivm_summary['coef']
plt.errorbar(iivm_summary.index, iivm_summary.coef, fmt='o', yerr=errors)
plt.ylim([0, 17500])

plt.title('Interactive IV Model (IIVM)')
plt.xlabel('ML method')
_ = plt.ylabel('Coefficients and 95%-CI')

### Summary of Results

To sum up, let's merge all our results so far and illustrate them in a plot.

In [ ]:
df_summary = pd.concat((plr_summary, irm_summary, iivm_summary)).reset_index().rename(columns={'index': 'ML'})
df_summary['Model'] = np.concatenate((np.repeat('PLR', 6), np.repeat('IRM', 6), np.repeat('IIVM', 6)))
print(df_summary.set_index(['Model', 'ML']))

In [ ]:
plt.figure(figsize=(10, 15))
for ind, model in enumerate(['PLR', 'IRM', 'IIVM']):
    plt.subplot(3, 1, ind+1)
    this_df = df_summary.query('Model == @model')
    errors = np.full((2, this_df.shape[0]), np.nan)
    errors[0, :] = this_df['coef'] - this_df['2.5 %']
    errors[1, :] = this_df['97.5 %'] - this_df['coef']
    plt.errorbar(this_df.ML, this_df.coef, fmt='o', yerr=errors,
                 color=face_colors[ind], ecolor=face_colors[ind])
    plt.ylim([0, 20000])
    plt.title(model)
    plt.ylabel('Coefficients and 95%-CI')

_ = plt.xlabel('ML method')

We report results based on six ML and DL methods for estimating the nuisance functions used in
forming the orthogonal estimating equations. Except the single hidden-layer MLP, which did not converge, we find that the estimates of the treatment effect are stable across ML methods. The estimates are highly significant, hence we would reject the hypothesis
that 401(k) participation has no effect on financial wealth.

# The S-Learner, T-Learner, and TARNet for Heterogenous Treatment Effects Estimation

Please refer to the paper (https://osf.io/preprints/socarxiv/aeszf)  regarding the core concepts in the notebook


Social scientists aim to understand how one variable causally influences another. Consider an example from Veitch et al., 2020: you have data from Reddit and want to determine how the author's stated gender (=T) influences the number of upvotes a post receives (=Y). There's a complexity, though: gender may influence the post's content (=X) — such as tone, style, or topics — which in turn can affect its upvotes. To isolate the direct causal impact of gender on upvotes, it's crucial to control for these content-related factors. Essentially, the goal is to discern if gender (=T) still affects upvotes (=Y) when assuming posts have similar content attributes.


The following tutorials are a gentle introduction to building deep learning models for causal inference using the selection on observables identification strategy. In particular, these model are designed to estimate the  average treatment effect (ATE) and the conditional average treatment effect (CATE). The ATE is defined as:

$$ATE =\mathbb{E}[Y(1)-Y(0)]$$

where $Y(1)$ and $Y(0)$ are the potential outcomes had the unit received or not received the treatment, respectively. The CATE is defined as,

$$CATE =\mathbb{E}[Y(1)-Y(0)|X=x]$$

where $X$ is the set of selected, observable covariates, and $x \in X$.

Because selection on observables is a simple identification strategy, these estimators are simple neural networks. This tutorial is thus also a gentle introduction to writing models in TensorFlow, and getting started coding deep learning models.

**These tutorials are for you if:**

1. You want a quick and dirty introduction to DL + Selection on Observables literature with minimal math, or...

2. You want a gentle introduction to writing and training custom models in Tensorflow 2 and...

3. You have a basic familiarity with causal inference and...

4. You have a basic familiarity with Python and object oriented programming.

**DISCLAIMER**: Before we get started, I want to make clear that the point of any of these pedagogical tutorials is not to argue that one of these models is empirically (or theoretically) straight-up superior to another. This is only one tiny simulation (from a [benchmark](https://arxiv.org/abs/2107.13346) that has itself been critiqued) without hyperparameter optimization. These are toy pedagogical programming examples, not benchmarking notebooks. We use the same specification and hyperparameters for all three models for comparison, but I'm sure you could get better results with each if you tweaked them. If you want to apply these to real data, you should try different things and do careful model selection.

----

## What are we doing here?

These model are designed to estimate the  average treatment effect (ATE) and the conditional average treatment effect(CATE) under a selection on observables identification strategy. The ATE is defined as:

$$ATE =\mathbb{E}[Y_i(1)-Y_i(0)]= \mathbb{E}[{\tau_i}]$$

where $Y_i(1)$ and $Y_i(0)$ are the potential outcomes had unit $i$ received or not received the treatment, respectively. The CATE is defined as,

$$CATE(x) =\mathbb{E}[Y_i(1)-Y_i(0)|X=x]$$

where $X$ is the set of selected, observable covariates, and $x \in X$.

Because selection on observables is a simple identification strategy, these estimators are simple neural networks. This tutorial is thus also a gentle introduction to writing models in TensorFlow, and getting started coding deep learning models.

## Why use deep learning for causal inference?

1. Appropriately built neural network models are among the **lowest bias** estimators in our statistical arsenal.

2. For similar reasons, the complex response surfaces learned by neural networks make them well-suited for estimating **heterogeneous treatment effects**.

4. Most excitingly, deep learning has the ability to allow us to control for confounding found in **complex data types like images, text, and networks**.

3. Although most of these models don't make theoretical guarantees, representation learning **might be more robust to empirical violations of overlap** than simpler adjustment strategies.

One more point: even if we cannot formally satisfy causal inference assumptions, these architectures are still very useful for **creating interpretable ML models** where we can isolate the contributions of specific covariates to predicting the outcome.

## Notation
**Causal identification**

- Observed covariates/features: $X$

- Potential outcomes: $Y(0)$ and $Y(1)$

- Treatment: $T$

- Average Treatment Effect: $ATE =\mathbb{E}[Y(1)-Y(0)]$

- Conditional Average Treatment Effect: $CATE =\mathbb{E}[Y(1)-Y(0)|X=x]$

**Deep learning estimation**

- Predicted outcomes: $\hat{Y}(0)$ and $\hat{Y}(1)$

- Outcome modeling functions: $\hat{Y}(T)=h(X,T)$

- Representation functions: $\Phi(X)$ (producing representations $\phi$)

- Loss functions: $\mathcal{L}(true,predicted)$, with the mean squared error abbreviated $MSE$ and binary cross-entropy as $BCE$

- Estimated CATE: $\hat{CATE}=(1-2t)(\hat{{y}}(t)-\hat{y}(1-t))$

- Estimated ATE: $\hat{ATE}=\frac{1}{n}\sum_{i=1}^n\hat{CATE_i}$


## Standard assumptions for causal identification under selection on observables
Standard assumptions for model-based causal inference apply here (from [Johansson et al., 2020](https://arxiv.org/pdf/2001.07426.pdf)):
1. **Conditional Ignorability/Exchangability**.The potential outcomes $Y(0)$, $Y(1)$ and the treatment $T$ are conditionally independent given $X$,
$$Y(0),Y(1)\perp \!\!\! \perp T|X $$
Conditional gnorability specifies that there are no *unmeasured confounders* that affect both treatment and outcome outside of those in the observed covariates/features $X$.


2. **Consistency/Stable Unit Treatment Value Assumption (SUTVA)**. Consistency specifies that when a unit recieves treatment, we observe the potential outcome. Moreover, the response of any unit does not vary with the treatment assignment to other units (i.e., no network effects), and the form/level of treatment is homogeneous and consistent across units,
$$T=t \rightarrow Y=Y(T)$$


3. **Overlap** In any context $x \in X$, any treatment $t\in \{0,1\}$ has a non-zero probability of being observed in the data,

$$\forall x \in X, t\in\{0,1\}:p(T=t|X=x)>0$$

Note that the overlap assumption does not require that the empirical data are necessarily balanced, but that the two treatment distributions have common support.

## Data

The IHDP dataset used in this example is a naturalistic simulation introduced in [Hill, 2011](https://www.tandfonline.com/doi/abs/10.1198/jcgs.2010.08162?casa_token=b8-rfzagECIAAAAA:QeP7C4lKN6nZ7MkDjJHFrEberXopD9M5qPBMeBqbk84mI_8qGxj01ctgt4jdZtORpu9aZvpVRe07PA) to evaluate estimation of heterogeneous treatment effects ($CATE$). The  25 covariates/features for the 747 units (139 treated) in the dataset were taken from an experiment, but Hill simulated the outcomes to create known counterfactuals. The data are available from Fredrik Johansson's website. IHDP is the de facto benchmark in this literature.

<details><summary>Additional details from Hill, 2011</summary>
<blockquote>[Hill] used experimental data from the Infant Health and Development Program (IHDP), a randomized experiment that began in 1985, targeted low-birth-weight, premature infants, and provided the treatment group with both intensive high-quality child care and home visits from a trained provider.... [The response surface] is nonlinear and not parallel across treatment conditions, with $Y(0)∼\mathcal{N}(exp((X+W)\beta_B),1)$ and $Y(1)∼\mathcal{N}(X\beta_B−\omega^s_B,1)$, where $W$ is an offset matrix of the same dimension as $X$ with every value equal to 0.5, $\beta_B$ is a vector of regression coefficients (0, 0.1, 0.2, 0.3, 0.4) randomly sampled with probabilities (0.6, 0.1, 0.1, 0.1,0.1). For the sth simulation, $\omega^s_B$ was chosen in the overlap setting, where we estimate the effect of the treatment on the treated, such that theconditional average treatment effect for the treated equals 4.</blockquote>
</details>

`y` is the simulated outcome that may represent $Y(0)$ or $Y(1)$ depending on `t`. Note that we rescale it here to improve convergence. `mu_0` and `mu_1` are "noiseless" potential outcomes where Hill simply used the mean of the normal distribution described in the spoiler.

There are 100 stochastic simulations in this data. For this example we will just use the eighth one.

In [ ]:
import numpy as np
!pip install scikit-learn==0.24.2
from sklearn.preprocessing import StandardScaler
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.train.npz
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.test.npz

def load_IHDP_data(training_data,testing_data,i=7):
    with open(training_data,'rb') as trf, open(testing_data,'rb') as tef:
        train_data=np.load(trf); test_data=np.load(tef)
        y=np.concatenate(   (train_data['yf'][:,i],   test_data['yf'][:,i])).astype('float32') #most GPUs only compute 32-bit floats
        t=np.concatenate(   (train_data['t'][:,i],    test_data['t'][:,i])).astype('float32')
        x=np.concatenate(   (train_data['x'][:,:,i],  test_data['x'][:,:,i]),axis=0).astype('float32')
        mu_0=np.concatenate((train_data['mu0'][:,i],  test_data['mu0'][:,i])).astype('float32')
        mu_1=np.concatenate((train_data['mu1'][:,i],  test_data['mu1'][:,i])).astype('float32')

        data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
        data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
        data['y']=data['y'].reshape(-1,1)
        #rescaling y between 0 and 1 often makes training of DL regressors easier
        data['y_scaler'] = StandardScaler().fit(data['y'])
        data['ys'] = data['y_scaler'].transform(data['y'])

    return data

data =load_IHDP_data(training_data='./ihdp_npci_1-100.train.npz',testing_data='./ihdp_npci_1-100.test.npz')

#concatenate t so we can use it as input
xt = np.concatenate([data['x'], data['t']], 1)

`data` is a dictionary (equivalent to a list in R). I'll print out a bit for you to see what it looks like:

In [ ]:
for key in data:
  if key == 'y_scaler': continue
  print(key, data[key][:5])

## Attempt Number 1: Pure Outcome Modeling (S[ingle]-Learner)

As a means to get our feet wet, we're going to start with the simplest way to estimate $\hat{CATE}$: a single multi-layer peceptron (sometimes called feed-forward or just deep neural network).

<figure><img src=https://github.com/kochbj/Deep-Learning-for-Causal-Inference/blob/main/images/Slearner.png?raw=true width="900"><figcaption><b>Fig 1: S-learner.</b> The S-learner is a deep feed-forward network or multilayer-percepetron. In a feed-forward neural network, additional fully connected (parameterized) layers of neurons are added between the inputs and output neuron. Purple indicates inputs, orange indicates network layers, and white indicates outputs. In this figure, the size of the hidden layers are first shown as nodes and then generically abstracted as boxes. The dashes between the orange shapes indicate an unspecifed number of additional hidden layers. The dashed lines on the right indicate non-gradient plug-in computations that occur after training. In causal inference settings, this architecture is sometimes called a S(ingle)-learner because one feed-forward network learns to predict both potential outcomes.</figcaption></figure>

To make this feasible, the network will take $X$ and $T$ as input and predict $Y$.

We'll label this function $h$ so $\hat{Y}=h(X,T)$.


We'll use the mean squared error as the loss,
$$\mathcal{L}(Y,h(X,T))=MSE(Y,h(X,T))=\frac{1}{n}\sum_{i=1}^n [h(x_i,t_i)-y_i]^2$$

Let's start by importing packages....

In [ ]:
!pip install -q tensorflow==2.8.0
import tensorflow as tf
import numpy as np #numpy is the numerical computing package in python
print(tf.__version__)

The next block specifies a function to build the model using Tensorflow 2's **Sequential API**. The **Sequential API** is the simplest of three API's in Tensorflow (see this [post](https://medium.com/tensorflow/what-are-symbolic-and-imperative-apis-in-tensorflow-2-0-dfccecb01021) for pros and cons). Most of the tutorial will be taught in the more powerful functional API, but I wanted to show you this first.

In words, this API is just taking a list of fully connected layers and creating an MLP. For now just ignore the other arguments beyond number of `units` in the layer; I'm just including all of these other specifications to make our S-learner comparable to our T-learner and TARNet.

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD

reg_l2=.01
s_learner = tf.keras.models.Sequential([
  Dense(units=200, activation='elu', kernel_initializer='RandomNormal'),
  Dense(units=200, activation='elu', kernel_initializer='RandomNormal'),
  Dense(units=200, activation='elu', kernel_initializer='RandomNormal'),
  Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2)),
  Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2)),
  Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2)),
])

Now let's specify the loss function...

In [ ]:
loss_fn = tf.keras.losses.MeanSquaredError() #specify the loss

In [ ]:
#@title Run this block. (Details we'll abstract away for now) { display-mode: "form" }
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD

val_split=0.2
batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)

sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        #40 is Shi's recommendation for this dataset, but you should tune for your data
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
    ]
#optimzier hyperparameters
sgd_lr = 1e-5
momentum = 0.9

Below are the two core methods we need to train our model.

`compile` creates a static computational graph of your network for training. At a minimum, you need to give it an optimizer and a loss function.

`fit` is your main training loop.

We'll pass through our data up to 300 times (may be less due to regularization conditions we'll discuss later), using batch sizes of 64. We reserve 20% of the data for validation. Ignore `sgd_callbacks` for now.

In [ ]:
s_learner.compile(optimizer=SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
                    loss=loss_fn,
                    metrics=loss_fn)

s_learner.fit(x=xt,y=data['ys'],
                validation_split=.2,
                epochs=300,
                batch_size=64,
                callbacks=sgd_callbacks,
                verbose=1)
print("Done")

### Estimating the ATE/CATE

Now we can estimate causal effects in either the whole dataset or a heldout testing sample. For simplicity, we just use the whole dataset here.  This is unorthodox in machine learning, but here we are interested in inference, not prediction.

Although our ultimate goal is to estimate the $CATE$, our loss function only minimizes the factual error to estimate $\hat{Y}$. This is a reflection of the fundamental problem of causal inference: we only observe one potential outcome for each unit. To get the quantities we want, we'll have to artificially toggle the treatment to get both $\hat{y}(t)$ and $\hat{y}(1-t)$ for each unit.


In [ ]:
import pandas as pd
import numpy as np
#create fake ones and zeros to feed network
zeros=np.expand_dims(np.zeros(data['x'].shape[0]),1)
ones=np.expand_dims(np.ones(data['x'].shape[0]),1)
x_untreated = np.concatenate([data['x'], zeros], 1)
x_treated = np.concatenate([data['x'], ones], 1)
y0_pred_slearner=s_learner.predict(x_untreated)
y1_pred_slearner=s_learner.predict(x_treated)

We can then plug in our predictions $\hat{Y}(0)$ and $\hat{Y}(1)$ to calculate the predicted CATE as

$$\hat{CATE_i}=(1-2t_i)(\hat{y_i}(t)-\hat{y_i}(1-t))$$
and the predicted average treatment effect as,
$$\hat{ATE}=\frac{1}{n}\sum_{i=1}^n\hat{CATE_i}$$

Since we know the true $CATE$s in our simulations, let's go over some commonly used evaluation metrics in this literature....

### Evaluation Metrics

Within this literature, it is common practice to evaluate model performance on simulations using the Precision Estimation of Heterogeneous  Effects ($PEHE$) from [Hill, 2011](https://www.tandfonline.com/doi/abs/10.1198/jcgs.2010.08162?casa_token=b8-rfzagECIAAAAA:QeP7C4lKN6nZ7MkDjJHFrEberXopD9M5qPBMeBqbk84mI_8qGxj01ctgt4jdZtORpu9aZvpVRe07PA). $PEHE$ measures the error in estimates of the $CATE$:

$$\sqrt{PEHE}=\sqrt{\frac{1}{N}\sum_{i=1}^N(CATE_i-\hat{CATE_i})^2}$$

The PEHE is more than just a metric, it has theoretical significance in literature in the definition of generalization bounds.

Since we know both potential outcomes in simulation we might also like to calculate bias in $\hat{ATE}$ and $\hat{CATE}$,
- $ATE_{bias} = |ATE-\hat{ATE}|$
- $CATE_{bias} = \frac{1}{N}\sum_{i=1}^N |CATE_i-\hat{CATE_i}|$

In [ ]:
def plot_cates(y0_pred,y1_pred,data):
  #dont forget to rescale the outcome before estimation!
  y0_pred = data['y_scaler'].inverse_transform(y0_pred)
  y1_pred = data['y_scaler'].inverse_transform(y1_pred)
  cate_pred=(y1_pred-y0_pred).squeeze()
  cate_true=(data['mu_1']-data['mu_0']).squeeze() #Hill's noiseless true values
  ate_pred=tf.reduce_mean(cate_pred)

  print(pd.Series(cate_pred).plot.kde(color='blue'))
  print(pd.Series(cate_true).plot.kde(color='green'))

  print(pd.Series(cate_true-cate_pred).plot.kde(color='red'))
  pehe=tf.reduce_mean( tf.square( ( cate_true - cate_pred) ) )
  sqrt_pehe=tf.sqrt(pehe).numpy()
  print("\nSQRT PEHE:",sqrt_pehe)
  print("Estimated ATE (True is 4):", ate_pred.numpy(),'\n\n')

  print("\nError CATE Estimates: RED")
  print("Individualized CATE Estimates: BLUE")
  print("Individualized CATE True: GREEN")
  return sqrt_pehe,np.abs(ate_pred.numpy()-4)

plot_cates(y0_pred_slearner,y1_pred_slearner,data)

### Analysis of S-learner results

Before we move on, let's actually LOOK at our results. It's clear that this model ran for 300 epochs and didn't learn anything at all! Take this finding with a major grain of salt; if you played around with the optimizer, capacity (number of layers and neurons), and other settings you might get this model to converge. This would be a good exercise! But do look at the disclaimer above. Let's keep going and see if we can find something more interesting.

----


## Attempt Number 2: T-Learner

The next most sophisticated thing we could do is fit two outcomes models independently, one for $Y(0)$ and one for $Y(1)$. This network is called a T-Learner.
<figure><img src=https://github.com/kochbj/Deep-Learning-for-Causal-Inference/blob/main/images/TLearner.png?raw=true width="900"><figcaption><b>Fig 2: T-learner.</b> The T-learner consists of two independent feed-forward networks learning the two different potential outcomes. Purple indicates inputs, orange indicates network layers, and white indicates outputs. The dashes between colored shapes indicate an unspecifed number of additional hidden layers. The dashed lines on the right indicate non-gradient, plug-in computations that occur after training.</figcaption></figure>

It would be pretty easy to implement the T-Learner as we did above using two Sequential API models. Instead we are going to transition to the **Functional API.** The Functional API is keras before it was absorbed into TF2. It is much easier to write complex models with the Functional API or Imperative (OOP) API.

### Coding up T-Learner

Okay, let's build the model! The rest of this tutorial basically modifies Claudia Shi's beautiful [implementation](https://github.com/claudiashi57/dragonnet) of TARNet from her [DragonNet paper](https://arxiv.org/pdf/1906.02120.pdf) (featured in a subsequent tutorial).

It's idiomatic in the functional API to declare a layer and immediately pass it's inputs so you can follow the forward-pass through the network. The only
layers that should be unfamiliar to you are the `Input` layer and `Concatenate` layer. Every graph is required to have an input layer to specify the dimensions of the input before compilation. `Concatenate` is just one of several utility layers in the API.

The model itself is still pretty simple: we use two output layers for each head with 100 neurons each. There are again a couple ways to have multiple outputs in the Functional API, but here we concatenate the two outputs into a list of vectors. We apply regularization to the output heads.

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model

def make_tlearner(input_dim, reg_l2):
    '''
    The first argument is the column dimension of our data.
    It needs to be specified because the functional API creates a static computational graph
    The second argument is the strength of regularization we'll apply to the output layers
    '''
    x = Input(shape=(input_dim,), name='input')

    #in TF2/Keras it is idiomatic to instantiate a layer and pass its inputs on the same line unless the layer will be reused
    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(x)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(x)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #a convenience "layer" that concatenates arrays as columns in a matrix
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions])
    #the declarations above have specified the computational graph of our network, now we instantiate it
    model = Model(inputs=x, outputs=concat_pred)

    return model

The `summary` method can be used to confirm that the architecture is specified correctly.

One of the advantages of the functional API is that you can also visualize static computational graphs (very similar to the cartoon representation above).

In [ ]:
tlearner_model=make_tlearner(25,.01)

print(tlearner_model.summary())
tf.keras.utils.plot_model(tlearner_model, show_shapes=True, show_layer_names=True, to_file='tlearner.png')

from IPython.display import Image # this just Jupyter notebook stuff
Image(retina=True, filename='tlearner.png')


<font color='red'><h2>Check Your Understanding:</h2></font>

What will happen if we use the same loss function as above?

<details><summary>Answer</summary>

The two heads are getting the same input data so they will be calculating the same gradients and will end up learning the same thing. We have two options:

A. Feed the two networks different data. This would essentially mean training the two networks independently or restructuring our data so that batch sizes of treated and control units can be split equally after input.

B. Somehow ensure that each head only receives error gradients for the correct treatment group. This will require writing a custom loss function.

Let's go with B.
</details>



### Specifying the loss function
There are again at least four different ways to specify loss functions in Tensorflow2: if you have a standard loss there are built-in options (as above), you can specify them as custom functions, custom objects, or build them into custom layers of your network. Here we've written a function.

 Note that we compute $\mathcal{L}(Y(0),h(X,0))$ and $\mathcal{L}(Y(1),h(X,1))$ separately and just add them to get the whole loss. Tensorflow will apply the gradients appropriately to the different outcome and representation layers.

In [ ]:
# every loss function in TF2 takes 2 arguments, a vector of true values and a vector predictions
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

### Training and Fitting the Model


<details><summary>A brief spoiler about training neural networks if you've never done so before.</summary>

When you use other types of machine learning models, optimization of the model parameters is typically done for you under the hood and you simply wait for training to finish. In contrast, neural networks have so many parameters that optimization becomes an art.

Rather than training on the whole training dataset at once, neural networks are trained on mini-batches of dozens to a few hundred examples. This is a compromise between applying error gradients from a single example (computationally expensive) and using the whole training dataset (expensive in terms of memory; may not work as well for losses that are not perfectly convex). The error gradient is applied to the network parameters after each mini-batch. A complete iteration through all mini-batches in the training set is called an **epoch.**

After each epoch we run prediction on the entire validation set. While there are a number of regularization techniques used in DL to prevent overfitting (norms, dropout, batch normalization), the most important is **early stopping.** To prevent overfitting, we wish to stop training after several consecutive epochs where the validation loss has failed to improve. The number of epochs to wait after early stopping is often called a *patience* hyperparameter.

The proportion of the gradient the optimizer backpropagates to the parameters is called the **learning rate.** A learning rate that is too small takes a long time to train. A learning rate that is too large will overshoot optima. Learning rate schedulers are used to adaptively slow the learning rate as you get closer to an optimum.

---

</details>


We will continue to use the builtin Keras `.fit` infrastructure for training the model which makes things super easy (you can of course write the training loop and calculate the gradients yourself). There are a lot of hyperparameter choices here, but I won't dwell on them because hyperparameter selection will be covered in the next tutorial.

Now let's get to the details I hid from you above!

 In this example we use stochastic gradient descent to optimize the model with an initial learning rate of 1E-4 and momentum of .9. You can also try other optimizers (e.g., ADAM). **While you should experiment with different learning rates, I recommend having a conservative (smaller) learning rate because we really want our estimator to be unbiased.**

 To avoid overfitting, we stop training deep learning models when the validation loss stops improving. In Tensorflow the `EarlyStopping` callback automatically stops training after a number of epochs with no improvement on the validation loss (`patience` parameter). The `ReduceLROnPlateau` adaptively lowers the learning rate of the optimizer as we approach validation loss plateaus so that the optimizer does not overshoot the current optimum.

We use a mini-batch size of 64. Other papers have recommmended batch sizes up to 200 with this dataset. **The batch size is an important consideration for these causal inference architectures because you really want to make sure each mini-batch has both treatment and control examples for the representation layers.** This is obviously less of a problem for datasets with high proportions of treated units.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD

val_split=0.2
batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([data['ys'], data['t']], 1) #we'll use both y and t to compute the loss


sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        #40 is Shi's recommendation for this dataset, but you should tune for your data
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
    ]
#optimzier hyperparameters
sgd_lr = 1e-5
momentum = 0.9
tlearner_model.compile(optimizer=SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
                    loss=regression_loss,
                    metrics=regression_loss)

tlearner_model.fit(x=data['x'],y=yt,
                callbacks=sgd_callbacks,
                validation_split=val_split,
                epochs=300,
                batch_size=batch_size,
                verbose=verbose)
print("DONE!")

Great! Let's calculate our causal effects and see what we get...

In [ ]:
concat_pred=tlearner_model.predict(data['x'])
#dont forget to rescale the outcome before estimation!
y0_pred_tlearner,y1_pred_tlearner = concat_pred[:, 0],concat_pred[:, 1]
plot_cates(y0_pred_tlearner,y1_pred_tlearner,data)

### Analyzing the T-learner results

This model actually converged! The $ATE_{pred}$ is around 3.54 and $\sqrt{PEHE}$ should be around .95 We can see that the estimated $CATE$ distribution has some bias towards lower treatment effects.

----

## Attempt 3: Representation learning as a balancing strategy (TARNet)

### Representation learning

A core concept in deep learning is the idea that artificial neural networks have the capacity to project a set of complex features $X$ into a useful vector space. When data are transformed into this space, we call the resulting tensor a **representation** ([Goodfellow, et al. 2016](https://www.deeplearningbook.org/contents/representation.html)) (you might also see the term "embedding"). For social scientists most comfortable with linear models, we can think about the parameters in each feed-forward layer of a deep neural network as capturing every possible interaction between the values produced by the previous layer. Tasking the network to minimize error on a relevant downstream task encourages it to adjust these interaction parameters to learn useful representations. We can also think about these representation layers as automatically extracting useful  latent covariates/features.

The key intuition in this literature is that we want to train neural networks to learn a representation function $\Phi(X)$ where the data are deconfounded/balanced in the representation space. In other words, the distributions of the representations $\Phi(X|T=0)$ and $\Phi(X|T=1)$ are similar.

<figure><img src=https://github.com/kochbj/Deep-Learning-for-Causal-Inference/blob/main/images/balancing.png?raw=true width="900"><figcaption><a href=https://github.com/maxwshen/iap-cidl>From Shen and Johansson talk 2018</a></figcaption></figure>

Note that $\Phi$ must, in theory, be an invertible function for the  ignorability and overlap assumptions to hold. By invertible we mean that there is an inverse function such that $\Phi^{-1}(\Phi(X))=X$.


### TARNet
To encourage balanced representations, [Shalit et al., 2017](http://proceedings.mlr.press/v70/shalit17a/shalit17a.pdf) propose a simple two-headed neural network called Treatment Agnostic Regression Network (TARNet). Each head models a separate outcome. One head learns the function $\hat{Y}(1)=h(\Phi(X),1)$, and the other head learns the function $\hat{Y}(0)=h(\Phi(X),0)$. Both heads backpropagate their gradients to shared representation layers that learn $\Phi(X)$. Again, the hope is that these representation layers will learn to balance the data because they are used to predict both outcomes.

<figure><img src=https://github.com/kochbj/Deep-Learning-for-Causal-Inference/blob/main/images/TARNet.png?raw=true width="900"><figcaption><b>Fig 3: TARNet.</b> This architecture, originally introduced in <a href=http://proceedings.mlr.press/v70/shalit17a/shalit17a.pdf>Shalit et al., 2017</a>, is a T-learner with shared representation layers. Purple indicates inputs, orange indicates network layers, other colors indicate output layers, and white indicates outputs. The dashes between colored shapes indicate an unspecifed number of additional hidden layers. The dashed lines on the right indicate non-gradient, plug-in computations that occur after training.</figcaption></figure>


Other than this architectural change, this has the same loss as the T-Learner:

$$\mathcal{L}(Y,h(\Phi(X),T))=MSE(Y,h(\Phi(X),T))=\frac{1}{n}\sum_{i=1}^n [h(\Phi(x_i),t_i)-y_i(t_i)]^2$$

 The complete objective for the network is to minimize the parameters of $h$ and $\Phi$ for all $n$ units in the training sample such that,

\begin{equation}
\min_{h,\Phi}\frac{1}{n}\sum_{i=1}^n \mathcal{L}(y_i(t_i),h(\Phi(x_i),t_i)) + \lambda \mathcal{R}(h)\end{equation}

where $\mathcal{R}(h)$ is a model complexity term (e.g., for $L_2$ regularization) and $\lambda$ is a hyperparameter chosen by the user.


### Coding up TARNet

Okay, let's build the model!


<font color='red'><h2>Check Your Understanding:</h2></font>

How can you modify the T-Learner to make it TARNet?

If you want to see the answer you can double click on the hidden TARNet block below. **Note that even if you don't want to look at the code, you need to run this block to proceed!**

You can put your attempt in the empty code block below:

In [ ]:
#@title <font color='red'>Answer:</font> Full TARNet model (Definitely read this in detail and run it!) { display-mode: "form" }
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model

def make_tarnet(input_dim, reg_l2):
    '''
    The first argument is the column dimension of our data.
    It needs to be specified because the functional API creates a static computational graph
    The second argument is the strength of regularization we'll apply to the output layers
    '''
    x = Input(shape=(input_dim,), name='input')

    # REPRESENTATION
    #in TF2/Keras it is idiomatic to instantiate a layer and pass its inputs on the same line unless the layer will be reused
    #Note that we apply no regularization to the representation layers
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #a convenience "layer" that concatenates arrays as columns in a matrix
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions])
    #the declarations above have specified the computational graph of our network, now we instantiate it
    model = Model(inputs=x, outputs=concat_pred)

    return model


Cool! Let's check out what the model looks like...

In [ ]:
tarnet_model=make_tarnet(25,.01)

print(tarnet_model.summary())
tf.keras.utils.plot_model(tarnet_model, show_shapes=True, show_layer_names=True, to_file='tarnet.png')

from IPython.display import Image # this just Jupyter notebook stuff
Image(retina=True, filename='tarnet.png')

### Aside: Imperative/Object Oriented Implementation

If you prefer OOP or would like to see what this model might look like in Pytorch you can check out the spoiler below...

<details><summary>Imperative API Implementation</summary>

 The same model above might look something like this in the imperative API:
```python
class TarNet(tf.keras.Model):
    def __init__(self,
                 input_dim,
                 name='tarnet',
                 regularization=.01,
                 **kwargs):
        super(TarNet, self).__init__(name=name, **kwargs)
        self.encoder1=Dense(units=200, activation='elu', kernel_initializer='RandomNormal')
        self.encoder2=Dense(units=200, activation='elu', kernel_initializer='RandomNormal')
        self.encoder3=Dense(units=200, activation='elu', kernel_initializer='RandomNormal')

        self.regressor1_y0 = Dense(units=100, activation='elu', kernel_regularizer=tf.keras.regularizers.l2(regularization))
        self.regressor2_y0 = Dense(units=100, activation='elu', kernel_regularizer=tf.keras.regularizers.l2(regularization))
        self.regressorO_y0 = Dense(units=1, activation=None, kernel_regularizer=tf.keras.regularizers.l2(regularization))

        self.regressor1_y1 = Dense(units=100, activation='elu', kernel_regularizer=tf.keras.regularizers.l2(regularization))
        self.regressor2_y1 = Dense(units=100, activation='elu', kernel_regularizer=tf.keras.regularizers.l2(regularization))
        self.regressorO_y1 = Dense(units=1, activation=None, kernel_regularizer=tf.keras.regularizers.l2(regularization))


    def call(self,inputs):
        x=self.encoder1(inputs)
        x=self.encoder2(x)
        phi=self.encoder3(x)

        out_y0=self.regressor1_y0(phi)
        out_y0=self.regressor2_y0(out_y0)
        y0=self.regressorO_y0(out_y0)

        out_y1=self.regressor1_y1(phi)
        out_y1=self.regressor2_y1(out_y1)
        y1=self.regressorO_y1(out_y1)

        concat=tf.concat([y0,y1,propensity],axis=1)
        return concat
```
</details>

### Training and Fitting the Model

Last time, we used a built-in MSE loss function, but this time we'll write it from scratch as a function. This is good practice for future tutorials.

In [ ]:
# every loss function in TF2 takes 2 arguments, a vector of true values and a vector predictions
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

Now we'll compile and train the model as we did for the T-learner.

In [ ]:
tarnet_model.compile(optimizer=SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
                    loss=regression_loss,
                    metrics=regression_loss)

tarnet_model.fit(x=data['x'],y=yt,
                callbacks=sgd_callbacks,
                validation_split=val_split,
                epochs=300,
                batch_size=batch_size,
                verbose=verbose)
print("DONE!")

### Estimating the ATE/CATE

In [ ]:
concat_pred=tarnet_model.predict(data['x'])
#dont forget to rescale the outcome before estimation!
y0_pred_tarnet,y1_pred_tarnet = concat_pred[:, 0],concat_pred[:, 1]
plot_cates(y0_pred_tarnet,y1_pred_tarnet,data)

### Analyzing TARNet Results
Compared to the T-learner, the distribution of predicted $CATE$s visually appears to be less biased. The $ATE$ and $\sqrt{PEHE}$ estimates are slightly more accurate as well. In the next tutorial, we'll focus on hyperparameter optimization to further zero in our models.

### Exploring Heterogeneity

Of course we can also break down these heterogeneous treatment effects to see if we can find any interesting patterns using, for example, Google's [Facet Dive](https://pair-code.github.io/facets/). This is just demonstrative since our covariates are meaningless in the simulation, but it's still cool. The Facet Dive is now built into TensorBoard.

In [ ]:
#@title Explore Heterogeneity Using the Facet Dive

data['cate_pred']=y1_pred_tarnet-y0_pred_tarnet
facet_df=pd.DataFrame(data['x'])
facet_df['t']=data['t']
facet_df['y']=data['y']
facet_df['cate_pred']=data['cate_pred']


# Display the Dive visualization for the training data.
from IPython.core.display import display, HTML

jsonstr = facet_df.to_json(orient='records')
HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))

# Thats it!

- We first learned the motivations of double/debiased learning method for causal estimation based on a DGP example.

- Based on the 401(k) dataset, we applied `DoubleML` for ATE and LATE estimation with six different types of ML/DL models.

- In this tutorial we wrote some simple causal estimators starting with the S-learner, moving up to the T-learner, and ending with TARNet.

- We learned how to write custom models using the sequential and functional APIs in TF2, as well as custom losses.

- We built a TARNet model and tested it on the IHDP data.



# Homework

In this notebook, we saw multiple ways to leverage neural networks for causal inference. In this homework you will explore the application of neural networks with your own data and questions regarding causal relationships.

**1)** Define key variables - T (treatment), Y (outcome), and X (covariates) - in your question about a causal relationship. For instance, if you want to determine how the author's stated gender influences the number of upvotes a post receives, T would be gender, the number of upvotes would be Y, and post's tone, style, or topics might be X.

In [ ]:
T = 'value' #@param {type:"string"}
Y = 'value' #@param {type:"string"}
X = 'value' #@param {type:"string"}

**2)** How would you use deep learning for the causal inference?

In [ ]:
causal_inference = 'value' #@param {type:"string"}

**3)** Using your own dataset, implement deep learning-based causal inference in any form in a way that is related to your project or research.

**4)** Interpret how the results from 3 support or reject your hypothesis about the causal relationship.

In [ ]:
interpretation = 'value' #@param {type:"string"}